In [69]:
import bentoml
from bentoml.io import JSON, Text
import pandas as pd
import random
import numpy as np
import requests
import json

In [70]:
# Load input CSV file to analyze its content
file_path = 'failure_data/health_index_augdata.csv'
health_index_data = pd.read_csv(file_path)

In [71]:
### A Godd Condition transformer
good_condition_transformer = health_index_data.iloc[0]
print(good_condition_transformer['TransformerID'])
print(good_condition_transformer['PerformanceMetrics'])

TxID_1872
Performance with a load of 327.90248545686734 KW and temperature 83.59766981598536°C. Stable operating conditions without any Incidents


## Sumarization Example with summarize_transformer_report.py service

In [72]:
import boto3
import json

def summarize_text(input_string):
    # Create a Bedrock client
    bedrock = boto3.client('bedrock-runtime')

    # Prepare the input payload
    payload = {
        "inputText": f"Summarize the following text: {input_string}",
        "textGenerationConfig": {
            "maxTokenCount": 300,
            "stopSequences": [],
            "temperature": 0.7,
            "topP": 1
        }
    }

    # Convert the payload to JSON string
    body = json.dumps(payload)

    # Invoke the model (using Amazon Titan Text model as an example)
    response = bedrock.invoke_model(
        modelId="amazon.titan-text-express-v1",
        contentType="application/json",
        accept="application/json",
        body=body
    )

    # Parse the response
    response_body = json.loads(response['body'].read())
    summary = response_body['results'][0]['outputText']

    return summary


In [73]:
# Example usage
input_json = {"string": """
Transformer ID: TX-5137
InstallationDate: 2009-05-21
OperatingTime: 39.416660829 years.
LoadCondition: Normal Load
PerformanceMetrics: Performance with a load of 1391.7672635240103 KW and temperature 64.64806835742223°C. Operating conditions were impacted by No incidents.

Condition Summary:
Health Index: 95.2
Based on the current dissolved gas analysis (DGA) and health index data, this transformer is expected to last an additional 19.0 years. 

Specific gases dissolved in the transformer insulating oil, could help identify potential faults or degradation. Current levels are:
- Hydrogen: 2845 ppm.
- Oxygen: 5860 ppm.
- Nitrogen: 27842 ppm.
- Methane: 7406 ppm.
- CO: 32 ppm.
- CO2: 1344 ppm.
- Ethylene: 16684 ppm.
- Ethane: 5467 ppm.
- Acethylene: 7 ppm.
- DBDS: 19.0 ppm.

Other Key indicators suggest that:
- Power factor: 1.0.
- Interfacial V: 45.
- Dielectric rigidity: 55.
- Water content: 0 ppm.

Calculated Gas ratios are:
- Carbon Monoxide to Hydrogen Ratio: 0.0112478032.
- Methane to Hydrogen Ratio: 2.6031634446.
- Ethylene to Hydrogen Ratio: 5.8643233743.
- Acetylene to Hydrogen Ratio: 0.0024604569.

Maintenance Information:
- Maintenance ID: MNT-29391
- Maintenance Type: Emergency Maintenance 
- Last Maintenance Date: 2023-11-02
- ReplacementHistory: Replaced major components

Environmental and Operational Factors:
- Transformer Load (KW): 1391.767263524
- Temperature (Celsius): 64.6480683574
- Humidity (%): 90.9032108218 %.
- Precipitation Level: 48.9464070009 mm.

Alerts & Incidents:
- Alerts: None.
- Incident Type: None.
- Incident Details: No incidents.

Current Status Summary: 
"""}


In [ ]:
summary = summarize_text(input_json["string"])
print("Summary:", summary)

In [ ]:
formatted_result = result.replace(". ", ".\n")
print(formatted_result)

In [13]:
report_filename = "output_data/" + good_condition_transformer['TransformerID'] + "_report.txt"
print(report_filename)

output_data/TX-5137_report.txt


In [14]:
final_report = report + formatted_result
print(final_report)

with open(report_filename, "w") as f:
    f.write(final_report)


Transformer ID: TX-5137
InstallationDate: 2009-05-21
OperatingTime: 39.416660829 years.
LoadCondition: Normal Load
PerformanceMetrics: Performance with a load of 1391.7672635240103 KW and temperature 64.64806835742223°C. Operating conditions were impacted by No incidents.

Condition Summary:
Health Index: 95.2
Based on the current dissolved gas analysis (DGA) and health index data, this transformer is expected to last an additional 19.0 years. 

Specific gases dissolved in the transformer insulating oil, could help identify potential faults or degradation. Current levels are:
- Hydrogen: 2845 ppm.
- Oxygen: 5860 ppm.
- Nitrogen: 27842 ppm.
- Methane: 7406 ppm.
- CO: 32 ppm.
- CO2: 1344 ppm.
- Ethylene: 16684 ppm.
- Ethane: 5467 ppm.
- Acethylene: 7 ppm.
- DBDS: 19.0 ppm.

Other Key indicators suggest that:
- Power factor: 1.0.
- Interfacial V: 45.
- Dielectric rigidity: 55.
- Water content: 0 ppm.

Calculated Gas ratios are:
- Carbon Monoxide to Hydrogen Ratio: 0.0112478032.
- Methane

In [10]:
final_report = report + formatted_result
with open("report.txt", "w") as f:
    f.write(final_report)

# print the final_report
print(final_report)


Transformer ID: TX-5137
InstallationDate: 2009-05-21
OperatingTime: 39.416660829 years.
LoadCondition: Normal Load
PerformanceMetrics: Performance with a load of 1391.7672635240103 KW and temperature 64.64806835742223°C. Operating conditions were impacted by No incidents.

Condition Summary:
Health Index: 95.2
Based on the current dissolved gas analysis (DGA) and health index data, this transformer is expected to last an additional 19.0 years. 

Specific gases dissolved in the transformer insulating oil, could help identify potential faults or degradation. Current levels are:
- Hydrogen: 2845 ppm.
- Oxygen: 5860 ppm.
- Nitrogen: 27842 ppm.
- Methane: 7406 ppm.
- CO: 32 ppm.
- CO2: 1344 ppm.
- Ethylene: 16684 ppm.
- Ethane: 5467 ppm.
- Acethylene: 7 ppm.
- DBDS: 19.0 ppm.

Other Key indicators suggest that:
- Power factor: 1.0.
- Interfacial V: 45.
- Dielectric rigidity: 55.
- Water content: 0 ppm.

Calculated Gas ratios are:
- Carbon Monoxide to Hydrogen Ratio: 0.0112478032.
- Methane

In [52]:
### A Bad Condition transformer
bad_condition_transformer = health_index_data.iloc[10]
print(bad_condition_transformer['TransformerID'])
print(bad_condition_transformer['PerformanceMetrics'])

TxID_6398
Performance with a load of 1309.8197130407605 KW and temperature 51.93916099537851°C. Stable operating conditions without any Incidents


## With OpenLLM

In [ ]:
with bentoml.SyncHTTPClient("http://localhost:3000") as client:
    result = client.summarize(
        json_string=good_condition_transformer.to_json()
    )
print(result)

In [53]:
with bentoml.SyncHTTPClient("http://localhost:3000") as client:
    result = client.summarize(
        json_string=bad_condition_transformer.to_json()
    )
print(result)

ServiceUnavailable: Server is not ready after 30 seconds

In [ ]:
formatted_result = result.replace(". ", ".\n")
print(formatted_result)

In [17]:
report_filename = "output_data/" + input_data['TransformerID'].iloc[0] + "_report.txt"
print(report_filename)

output_data/TX-5137_report.txt


In [16]:
final_report = report + formatted_result
with open("report.txt", "w") as f:
    f.write(final_report)

# print the final_report
print(final_report)


Transformer ID: TX-5137
InstallationDate: 2009-05-21
OperatingTime: 39.416660829 years.
LoadCondition: Normal Load
PerformanceMetrics: Performance with a load of 1391.7672635240103 KW and temperature 64.64806835742223°C. Operating conditions were impacted by No incidents.

Condition Summary:
Health Index: 95.2
Based on the current dissolved gas analysis (DGA) and health index data, this transformer is expected to last an additional 19.0 years. 

Specific gases dissolved in the transformer insulating oil, could help identify potential faults or degradation. Current levels are:
- Hydrogen: 2845 ppm.
- Oxygen: 5860 ppm.
- Nitrogen: 27842 ppm.
- Methane: 7406 ppm.
- CO: 32 ppm.
- CO2: 1344 ppm.
- Ethylene: 16684 ppm.
- Ethane: 5467 ppm.
- Acethylene: 7 ppm.
- DBDS: 19.0 ppm.

Other Key indicators suggest that:
- Power factor: 1.0.
- Interfacial V: 45.
- Dielectric rigidity: 55.
- Water content: 0 ppm.

Calculated Gas ratios are:
- Carbon Monoxide to Hydrogen Ratio: 0.0112478032.
- Methane

### Causal language modeling for text generation via service llm_service.py (Hugging Face transformers)

In [ ]:
with bentoml.SyncHTTPClient("http://localhost:3000") as client:
    result = client.generate(
        json_string=health_index_data.iloc[1].to_json()
    )

print(result)

In [ ]:
import requests

url = "http://localhost:3000/generate_text"
input_data = {"json_string": health_index_data.iloc[1].to_json()} 
response = requests.post(url, json=input_data)
print(response.json())


In [105]:
# convert ouput JSON to a paragraph
def convert_json_to_paragraph(json_data):
  paragraph = ""
  for key, value in json_data.items():
    paragraph += f"{key}: {value} "
  return paragraph.strip()

In [106]:
paragraph = convert_json_to_paragraph(response.json())
print(paragraph)

generated_text: 
Transformer ID: TX-1873
InstallationDate: 2002-09-27
OperatingTime: 35.26853756
LoadCondition: Normal Load
PerformanceMetrics: Performance stable with a load of 573.4742788867065 KW and temperature 92.92760626605455°C
Condition Summary:
Health Index: 63.4
Predicted Remaining Life Expectancy: 6.0 years 

Specific gases dissolved in the transformer insulating oil, could help identify potential faults or degradation. Current levels are:
- Hydrogen: 4
- Oxygen: 26100
- Nitrogen: 60600
- Methane: 1
- CO: 206
- CO2: 1440
- Ethylene: 13
- Ethane: 0
- Acethylene: 13
- DBDS: 5.0

Other Key indicators suggest that:
- Power factor: 1.32
- Interfacial V: 40
- Dielectric rigidity: 56
- Water content: 4

Calculated Gas ratios are:
- Carbon Monoxide to Hydrogen Ratio: 51.5
- Methane to Hydrogen Ratio: 0.25
- Ethylene to Hydrogen Ratio: 3.25
- Acetylene to Hydrogen Ratio: 3.25

Maintenance Information:
- Maintenance ID: MNT-26380
- Maintenance Type: Preventive Maintenance 
- Last Main